# Reconfiguring the model for a task
In this notebook, we will have a look at how to interact with model and task configurations, and how to reconfigure the model for a certain task in the project.

In [ ]:
# As usual we will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server

from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

#### Selecting a project
Let's get a list of projects in the workspace, and select the project `COCO multitask animal demo` that we created in notebook [004 create pipeline project from dataset](004_create_pipeline_project_from_dataset.ipynb). Feel free to use a different project by changing the `PROJECT_NAME` variable

In [ ]:
PROJECT_NAME = "COCO multitask animal demo"
projects = project_client.list_projects()

project = project_client.get_project(PROJECT_NAME)

## Configuring a task in the project

#### Setting up the ConfigurationClient
To be able to inspect and change the project's configuration, we will need to set up a ConfigurationClient for the project.

In [ ]:
from geti_sdk.rest_clients import ConfigurationClient

configuration_client = ConfigurationClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

#### Selecting the task to configure
The project `COCO multitask animal demo` has two tasks: let's select the first one as an example

In [ ]:
task = project.get_trainable_tasks()[0]
print(task.summary)

#### Getting the task configuration
With the configuration_client and task ready, we can request the configuration for the task

In [ ]:
task_configuration = configuration_client.get_task_configuration(task_id=task.id)
print(task_configuration.summary)

#### Inspecting individual parameters
As you may know the parameters in the configuration contain much more metadata than what is shown in the summary above. We can have a closer look at the metadata for the individual parameters by accessing them directly (by their name) from the task_configuration. The cell below shows how to so

In [ ]:
print(task_configuration.batch_size.overview)

### Changing the task configuration values
The method `configuration_client.set_configuration` allows us to reconfigure the project and tasks. 

We can change the parameter values in the task_configuration that we obtained above, and then send it to the `set_configuration` method to update the configuration on the platform. So let's change a couple of values now first

In [ ]:
# Set batch size to 10
task_configuration.batch_size.value = 10

In [ ]:
# Double the learning rate
old_learning_rate = task_configuration.learning_rate.value
task_configuration.learning_rate.value = 2 * old_learning_rate

## Setting the configuration on the platform
With the values updated, we can now use the configuration_client to send the new task_configuration to the platform

In [ ]:
configuration_client.set_configuration(task_configuration)

#### Checking the new configuration, to make sure that it worked
In the cell below we get the configuration for the task again, and print a summary. You should see the changes in the parameter values reflected in the summary.

In [ ]:
new_task_configuration = configuration_client.get_task_configuration(task_id=task.id)
print(new_task_configuration.summary)